In [13]:
import re
import os
from glob import glob

from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 0
MODEL_NAME  = 'gpt-4o-mini'
EMBEDDING_NAME = 'text-embedding-3-large'

COLLECTION_NAME = 'korean_history'
PERSIST_DIRECTORY= 'vector_store/korean_history_db'
name = ["고대", "고려", "근대", "조선", "현대"]

In [3]:
# Split
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name=MODEL_NAME,
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
)

In [5]:
for i in name:
    file = glob(f"./pdf/인물/{i}/*.pdf")
    document_list = []
    for path in file:
        loader = PyMuPDFLoader(path)
        load_docs = loader.load()
        full_text = [doc.page_content for doc in load_docs] 
        full_text = ''.join(full_text)
        full_text = re.sub(r"관련사료", "", full_text)
        full_text = re.sub(r"\([一-龥]+\)", "", full_text)
        full_text = re.sub(r"\n", "", full_text)

        docs = splitter.split_text(full_text)
        id = "_".join(os.path.splitext(os.path.basename(path))[0].split('_')[:2])
        title = os.path.splitext(os.path.basename(path))[0].split('_')[-1]
        metadata = {
        "id":id,
        "title": title,
        "full_text": full_text
        }

        for doc in docs:
            _doc = Document(metadata=metadata, page_content=doc)
            document_list.append(_doc)

    embedding_model = OpenAIEmbeddings(model=EMBEDDING_NAME)

    vector_store = Chroma.from_documents(
        documents=document_list,
        embedding=embedding_model,
        ids=[doc.metadata['title'] for doc in document_list],
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
    )

    print(len(document_list))

72
85
88


c:\Users\Playdata\SKN06\단위 프로젝트\Prj03\prj03\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:322: UserWarning: Warning: Empty content on page 3 of document ./pdf/인물/조선\인물_조선_이제마.pdf
  warnings.warn(


155
6


In [8]:
embedding_model = OpenAIEmbeddings(model=EMBEDDING_NAME)
vector_store = Chroma(collection_name=COLLECTION_NAME, persist_directory=PERSIST_DIRECTORY, embedding_function=embedding_model)

In [9]:
vector_store._collection.count()

668

In [10]:
retriever = vector_store.as_retriever(search_type="mmr")

In [11]:
query = "장영실에 대해 알려줘"
result = retriever.invoke(query)
result

[Document(metadata={'full_text': '장영실[蔣英實]물시계, 해시계, 우량계 출신 배경장영실(蔣英實, 생몰년 미상)은 조선 초기의 과학기술자로, 본관은 아산이다. 성종대에 편찬된 『신증동국여지승람』에는 아산의 성씨 중에 ‘장씨’가 실려 있으며,  『아산장씨세보』에는 장영실이 아산 장씨의 시조 장서의 9세손으로 기록되어 있다.장영실의 아버지는 중국에서 귀화한 인물이고, 어머니는 기녀였다. 그런데 장영실의 아버지가언제 귀화했는가에 대해서는 『세종실록』과 『아산장씨세보』의 기록 사이에 차이가 나타난다. 먼저 『세종실록』에는 장영실의 아버지가 본래 원나라의 소주·항주 지역 사람이고어머니는 기생이라고 기록되어 있다. 즉, 장영실의 아버지는 중국에서 출생한 후 조선으로 귀화했다는 것이다. 반면, 『아산장씨세보』에서는 장영실의 9대조인 장서가 고려에 귀화하여 아산 장씨의 시조가 된것으로 기록하고 있다. 즉, 장서는 송나라 사람으로 대장군을 역임했으며, 금나라가 송나라를 공격했을 때 송나라 조정이 주전론과 주화론의 분열로 어지러워지자 중국을 떠나 고려로 이주하여 아산에 정착했다고 한다. 또, 장영실의 부친 장성휘는1전서를 역임했던 것으로 기록되어 있다. 이상과 같이 장영실의 선대가 중국에서 귀화했다는 것은 모두 인정하는 바이지만, 귀화시기에 대해서는 실록과 『아산장씨세보』 사이에 차이가 있다. 하지만 장영실의 가계에 관한 더 이상의 자료가 없기 때문에 어느 쪽이 정확하다고확언하기는 어렵다.장영실이 어린 시절에 어떤 성장 과정을 거쳤는지에 대해서는 알려진 바가 없다. 다만, 『연려실기술』에서 1421년(세종 3) 세종이 장영실을 불러 선기옥형 제도를 연구하게 했을 당시 장영실의 신분을 ‘동래의 관노’를 기록한 것을 볼 때, 그의 본래 신분이관노, 즉 천인이었음을 알 수 있다. 그가 어떤 경위로 관노가 되었는지는 알 수 없지만, 아마도 그의 어머니가 기녀였기 때문에 장영실도 어머니의 신분을 따라 천인이 됐던 것이 아닐까 추정된다. 뛰어난 기술로 신분의 굴

In [12]:
result = vector_store.similarity_search_with_score(
    query="법왕에 대해 알려줘",
    k=5
)
result

[(Document(metadata={'full_text': '법왕[法王]불법으로 왕권을 강화하다미상 ~ 600년(법왕 2) 개요법왕은 백제의 제29대 왕으로 재위 기간은 599년~600년이다. 즉위하자마자 살생을 금하는 명을 내렸다. 이듬해에는 왕흥사를 창건하면서 30명이 승려가 되는 것을 허가하였다.법왕은 2년이라는 짧은 재위기간에도 불구하고 불교 윤리를 확대시키기 위해 노력했고, 이를 통해 왕권을 강화하고자 했다. 법왕의 즉위 과정에 대한 논란법왕의 이름은 선 또는 효순이고, 성은 부여씨이다. 대부분의 문헌에서 제28대혜왕의 맏아들이라고 기록되어 있으나, 『수서』에는 제27대 위덕왕의 아들이라는 기록도 있다. 어머니와 왕비에 대해서는 알려진 바가 없다. 아들은 부여장인데, 후에 무왕이 되었다. 무왕이 조카인 복신을 당에 사신으로 보냈다는 기록으로 보아, 무왕 외에 또 다른 아들이나 딸이 있었을 것으로 추정된다. 고령의 혜왕이 재위 2년 만인 599년(혜왕 2)에 죽자, 법왕이 왕위에 올랐다. 그러나 법왕 역시 즉위한 이듬해 여름에 죽고 말았다.시호는 법이다. 법왕의 즉위과정에 대해서는 논란이 있는데, 첫 번째 주장은 다음과 같다. 498년(위덕왕 45), 위덕왕은 수에 사신을 보내 수가 고구려를 공격하면 백제가 향도가 되겠다고 자청하였다. 그러자 고구려가 이를 알고 백제 변경을 쳐들어와 약탈을 자행했다. 이때는 고령의 위덕왕이 사망 3개월 전으로, 왕의 통치력이 미약했을 시기이다. 당시 위덕왕의 동생인 혜왕 역시 고령이었고, 아들인 아좌태자는 왜에 머무르고 있었다. 그러므로 고구려전을 주도한 사람은 혜왕의 아들이자 위덕왕의 조카인 법왕이었다. 이 과정에서 법왕은 일종의 정변을 통해 정국 운영의주도권을 장악했다. 그리고 위덕왕이 죽자 왕위계승서열이 앞서는 자신의 아버지 즉, 혜왕을 명목상 즉위시켜 귀족들의 반발을 피했다. 그리고 혜왕이 죽자 자연스럽게 법왕이 즉위하게 되었다.반면에 이러한 주장을 반박하면서 위덕왕대 혜왕의 활약상을 주목하기도 한다. 혜왕이 

In [14]:
# Prompt Template 생성
messages = [
        ("ai", """
    당신은 한국사 전문가입니다. 주어진 context를 참고하여 한국어로 답변해주세요. 문서에 없는 내용은 답변할 수 없습니다. 모른다고 하세요.
{context}"""),
        ("human", "{question}"),
    ]
prompt_template = ChatPromptTemplate(messages)

# 모델
model = ChatOpenAI(model="gpt-4o-mini")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련 문서 조회) -> prompt_template(prompt 생성) model(정답) -> output parser
chain = {"context":retriever, "question":RunnablePassthrough()} | prompt_template | model | parser

In [17]:
result = chain.invoke("나옹혜근에 대해 알려줘")

In [18]:
print(result)

모른다고 하세요.
